# Central Limit Theorem

El **teorema del límite central** es un concepto estadístico que está en el foco de mucha de la ciencia de datos, en especial la que se dedica a realizar experimentación y la creación de modelos predictivos.

El *Central Limit Theorem* nos dice que:

 > Dado un conjunto de datos, cuya distribución es desconocida, los promedios de muestras tomadas de este conjunto se aproximarán a una distribución normal. Admás, el promedio de los promedios de todas las muestras será aproximadamente igual al promedio de la distribución original.
 
Sí, suena muy complicado... creo que es mejor explicarlo con una pequeña simulación.

## Simulación

Antes de comenzar, quiero dejar en claro la diferencia entre esta simulación y la vida real:

 - En la simulación tenemos acceso a la población completa. **En la vida real muy rara vez se tiene acceso a la población completa.**
 - En la simulación podemos tomar tantas muestras como queramos. **En la vida real muchas veces tenemos acceso a una sola muestra**.

Con esto fuera del camino, podemos comenzar.

### Científico Pokemon!

Imagina que te han encargado **encontrar cuál es el peso promedio de un Pokémon**. No suena a una tarea sencilla, ya que hay muchísimos (por el momento digamos que hay $1,000,000$) y atraparlos a todos para pesarlos no es una tarea sencilla, sin embargo el teorema del límite central nos puede ayudar.

Para nuestra simulación tendremos todos los pesos de los pokémon en la variable `population` (recuerda que tenemos acceso a toda la población porque esta es una simulación).

In [ ]:
// Algunos paquetes y using necesarios...
#r "nuget:MathNet.Numerics, 4.9.0"
#r "nuget:NumSharp, 0.20.4"
    
using System;
using NumSharp;
using MathNet.Numerics.Distributions;
using System.Linq;
using System.Collections;
using System.Collections.Generic;
using XPlot.Plotly;
using static XPlot.Plotly.Graph;
using MathNet.Numerics;
using KernelDensity = MathNet.Numerics.Statistics.KernelDensity;
using Histogram = MathNet.Numerics.Statistics.Histogram;

In [ ]:
NDArray ToNDArray(IEnumerable<double> data, int populationSize)
{
    return np.array(data.Take(populationSize).ToArray());
}

NDArray GetPokemonPopulation(int populationSize)
{
    var norm1 = ToNDArray(Normal.Samples(2, 2), populationSize);
    var norm2 = ToNDArray(Normal.Samples(15, 1), populationSize);
    norm1 = norm1 - norm1.min();
    var combined = np.concatenate(new[] { norm1, norm2 });
    var uniform = ToNDArray(ContinuousUniform.Samples(0, 5), 2 * populationSize);

    return uniform + combined;
}

var population = GetPokemonPopulation(10000);
// TODO: Print stats

In [ ]:

NDArray Kde(NDArray data, NDArray xaxis)
{
    int points = xaxis.shape[0];
    var samples = data.ToArray<double>();
    var xs = xaxis.ToArray<double>();
    var kde = np.zeros(points);
    for (int i = 0; i < xs.Length; i++)
    {
        var y = KernelDensity.EstimateGaussian(xs[i], 0.5, samples);
        kde.SetAtIndex(y, i);
    }
    return kde;
}

Scatter KdePlot(NDArray data, Histogram histogram)
{
    var kdex = np.linspace(histogram.LowerBound, histogram.UpperBound, 100, typeCode: NPTypeCode.Double);
    var kdey = Kde(data, kdex);
    return new Scatter
    {
        x = kdex,
        y = kdey,
        name = $"KDE"
    };
}

Tuple<Bar, NDArray, NDArray> HistPlot(NDArray data, Histogram histogram)
{
    var bins = histogram.BucketCount;
    var classWidth = (np.max(data) - np.min(data)).GetDouble() / bins;
    var xHist = np.zeros(bins);
    var yHist = np.zeros(bins);

    for (int i = 0; i < bins; i++)
    {
        var bucket = histogram[i];
        xHist.SetAtIndex((bucket.LowerBound + bucket.UpperBound) / 2, i);
        yHist.SetAtIndex(bucket.Count, i);
    }
    yHist = yHist / np.sum(yHist) / classWidth;

    var barPlot = new Bar() { x = xHist, y = yHist, name="" };

    return new Tuple<Bar, NDArray, NDArray>(barPlot, xHist, yHist);
}

PlotlyChart CreateChart(NDArray data, string name, int bins)
{
    var mean = data.mean().GetAtIndex<double>(0);
    var histogram = new Histogram(data.ToArray<double>(), bins);

    (Bar histPlot, NDArray xHist, NDArray yHist) = HistPlot(data, histogram);
    var kdePlot = KdePlot(data, histogram);

    var meanLine = new Scatter
    {
        x = new[] { mean, mean },
        y = new[] { 0, yHist.max<double>() },
        name = $"Mean {mean::0.####}"
    };

    var layout = new Layout.Layout
    {
        title = name,
        xaxis = new Xaxis { title = name },
        yaxis = new Yaxis { title = "Frequency" },
        bargap = 0
    };

    var chart = Chart.Plot(new List<Trace> { histPlot, meanLine, kdePlot });
    chart.WithLayout(layout);

    return chart;
}

display(CreateChart(population, "Distribution", 40))

### El teorema al rescate  

Imagina que tomas una muestra (`sample`) de... digamos 30 pokémon, y tomas el promedio de esta:

In [ ]:
var sampleSize = 30;
double SampleMean(NDArray data, int sampleSize)
{
    var shape = new NumSharp.Shape(sampleSize);
    var sample = np.random.choice(data, shape: shape);
    return sample.mean();
}

SampleMean(population, sampleSize)

Puede que el promedio de la muestra esté cerca del verdadero promedio de la población... ¿así que qué pasa si **tomas otras tantas muestras (`numSamples`), sacas el promedio de cada una y creas una grafica de distribución con ellas**? a esto último se le conoce en inglés como *sample means distribution*, al final de cuentas vamos a terminar con otra población de promedios de muestras en nuestra variable `sampleMeans`, y como tal la podemos graficar:

In [ ]:
var numSamples = 1000;

NDArray GetSampleMeans(NDArray data, int numSamples, int sampleSize)
{
    var means = np.zeros(numSamples);
    var shape = new NumSharp.Shape(sampleSize);
    for (int i = 0; i < numSamples; i++)
    {
        var sample = np.random.choice(data, shape: shape);
        means.SetAtIndex<double>(sample.mean(), i);
    }
    return means;
}

var sampleMeans = GetSampleMeans(population, numSamples, sampleSize);

display(CreateChart(sampleMeans, "Distribution", 40))

Si miras nuevamente la gráfica, podrás notar que tiene **una forma bastante conocida**: la de una campana, sí, como **una distribución normal**. Y algo aún más notable, el promedio de es aproximadamente igual al promedio de nuestra distribución original.

No importa qué distribución inicial tomemos, si seguimos el procedimiento de tomar muestras y sacar su promedio, terminaremos con una curva de distribución normal, cuyo promedio será muy aproximado al de la distribución original.

Un poco más formal:

 > Si tomamos muestras de tamaño n de una población y calculamos el promedio de cada una de esas muestras, no importa la forma de la distribución original de la población, la distribución de promedios seguirá una distribución normal.  
 
### Volviendo a los pokémon

Este teorema nos permite, con ayuda de la estadística, a tomar una sola muestra y usar el promedio de ella para poder aproximar (con cierto grado de error) el promedio de la población original.

## Más ejemplos  

Para probar el teorema, podemos tomar como base diferentes distribuciones: